# Esta tarea fue realizada en equipo por Betsy Torres y Mariana Briones

# Tarea 3.1
    
Compre una acción inicial que no pague dividendos y estime los resultados esperados usando la Teoría de las finanzas corporativas del factor X estocástico. Asuma una distribución normal. 
- Método analítico 
- Simule en python para probar la precisión.

### Importar librerias

In [1]:
import time
import scipy.stats as st
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
#import warnings
import numpy as np

### Funciones para ajustar distribuciones

In [2]:
def best_fit_distribution(data, bins=200, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

  # Distributions to check
    DISTRIBUTIONS = [        
        st.gennorm,st.genexpon,st.lognorm,st.lomax,st.maxwell,st.mielke,st.nakagami,st.ncx2,st.ncf,
        st.nct,st.norm,st.powerlognorm, st.uniform
    ]

    # Best holders
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf

    # Estimate distribution parameters from data
    for distribution in DISTRIBUTIONS:

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))

                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                
                except Exception:
                    pass

                # identify if this distribution is better
                if best_sse > sse > 0:
                    best_distribution = distribution
                    best_params = params
                    best_sse = sse

        except Exception:
            pass

    return (best_distribution.name, best_params)

def make_pdf(dist, params, size=10000):
    """Generate distributions's Probability Distribution Function """

    # Separate parts of parameters
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]

    # Get sane start and end points of distribution
    start = dist.ppf(0.01, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.01, loc=loc, scale=scale)
    end = dist.ppf(0.99, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.99, loc=loc, scale=scale)

    # Build PDF and turn into pandas Series
    x = np.linspace(start, end, size)
    y = dist.pdf(x, loc=loc, scale=scale, *arg)
    pdf = pd.Series(y, x)

    return pdf

### Obtención de Beta y parametros de distribución normal

In [9]:
Raw_nas = pd.read_csv('^IXIC.csv')
nas = Raw_nas['Adj Close']
nas = nas[1:].values/nas[:-1].values -1
nas = pd.DataFrame(nas,columns=['Adj Close'])

Raw_sp = pd.read_csv('SPOT.csv')
sp = Raw_sp['Adj Close']
sp = sp[1:].values/sp[:-1].values -1
sp = pd.DataFrame(sp,columns=['Adj Close'])

Beta = nas['Adj Close'].corr(sp['Adj Close'])
print('Beta: ' + str(Beta))
best_fit_sp, sp_params = best_fit_distribution(sp, 200)
print('SPOT ~ N(' + str(sp_params[0]*252) + ', ' + str(sp_params[1]*252**0.5) + ')')
best_fit_nas, nas_params = best_fit_distribution(nas, 200)
print('NASDAQ ~ N(' + str(nas_params[0]*252) + ', ' + str(nas_params[1]*252**0.5) + ')')

print('\n\n')

Beta: -0.044918330261616256
SPOT ~ N(0.0, 15.874507866387544)
NASDAQ ~ N(0.0, 15.874507866387544)





In [10]:
best_fit_sp, sp_params

('norm', (0.0, 1.0))

In [11]:
best_fit_nas, nas_params

('norm', (0.0, 1.0))

### Resolver analíticamente

$$ r =  \alpha + \beta (m - rf) + rf $$

$$ E[r] = E[\alpha + \beta (m - rf) + rf] $$
$$ E[r] = E[\alpha + \beta (m - rf) + rf] $$
$$ E[r] = E[\alpha] + E[\beta (m - rf)] + E[rf] $$
$$ E[r] = E[\alpha] + \beta E[m - rf] + E[rf] $$
$$ E[r] = E[\alpha] + \beta E[m] - \beta E[rf] + E[rf] $$
$$ E[r] = E[\alpha] + \beta E[m] - \beta * rf + rf $$

$$ r =  \alpha + \beta (m - rf) + rf $$

$$ Var[r] = Var [\alpha + \beta (m - rf) + rf] $$
$$ Var[r] = Var[\alpha] + Var[\beta (m - rf)] + Var[rf] + 2 *Cov(\alpha,m)$$
$$ Var[r] = Var[\alpha] + \beta^2 Var[m - rf] + Var[rf] + 2 *Cov(\alpha,m)$$
$$ Var[r] = Var[\alpha] + \beta^2 Var[m] - \beta^2 Var[rf] + Var[rf] + 2 *Cov(\alpha,m)$$
$$ Var[r] = Var[\alpha] + \beta^2 Var[m] - \beta^2 * 0 + 0 + 2 *Cov(\alpha,m)$$
$$ Var[r] = Var[\alpha] + \beta^2 Var[m] + 2 *Cov(\alpha,m)$$
$$ Var[r] = Var[\alpha] + \beta^2 Var[m] + 2 * \sqrt{Var[\alpha]*Var[m]}*Corr[\alpha,m]$$



### Resolver con simulación

In [7]:
t0 = time.time()
n=10000

rf = 0.03
alfa = pd.DataFrame(np.random.normal(loc = sp_params[0]*252, scale = np.sqrt(sp_params[1]*np.sqrt(252)), size=n))
alfa.columns = ['Alfa']

m = pd.DataFrame(np.random.normal(loc = nas_params[0]*252, scale = np.sqrt(nas_params[1]*np.sqrt(252)), size=n))
m.columns = ['Market']

R=pd.DataFrame(np.zeros(n))
i=0
for i in range(n):
    R.iloc[i,0]=alfa.iloc[i,0] + Beta*m.iloc[i,0]-Beta*rf+ rf

print("Alfa (spot)")
print(alfa.mean().values)
print(alfa.var().values)

print("m (ixic)")
print(m.mean().values)
print(m.var().values)

print("R")
print(R.mean().values)
print(R.var().values)


t1 = time.time()
print('Elapsed time: ' + str(np.round(t1-t0, 3)) + ' sec.')

Alfa (spot)
[-0.05059922]
[16.18162989]
m (ixic)
[0.00481483]
[16.03777769]
R
[-0.01946794]
[16.1809358]
Elapsed time: 8.133 sec.


In [8]:
SumZ = alfa.Alfa + m.Market
z = pd.DataFrame(np.cov(alfa, m, rowvar=0)).iloc[0,1] #-0.8*.03+0.03
#c = np.cov(z.T)